In [148]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline


In [149]:
df = pd.read_csv('training_dataset_V3.csv')
df.describe()

,Unnamed: 0,year,id_driver,num_trucks,days_signup_to_approval,loads,marketplace_loads_otr,marketplace_loads_atlas,marketplace_loads,brokerage_loads_otr,brokerage_loads_atlas,brokerage_loads,total_loads
count,83414.000000,83414.000000,83414.000000,83344.000000,71124.000000,83414.000000,83414.000000,83414.000000,83414.000000,83414.000000,83414.000000,83414.000000,83414.000000
mean,41706.500000,2018.960930,18222.414954,22.582921,298.752489,2.076270,29.477762,71.579675,101.057436,148.258422,13.073021,161.331443,266.502661
std,24079.692014,1.359343,11667.704926,48.829719,390.345107,2.672163,88.171940,194.532776,214.502147,415.978060,42.241592,413.792137,448.806175
min,0.000000,2015.000000,20.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,20853.250000,2018.000000,7890.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,37.000000
50%,41706.500000,2019.000000,16299.000000,4.000000,61.000000,1.000000,2.000000,0.000000,13.000000,15.000000,0.000000,37.000000,110.000000
75%,62559.750000,2020.000000,28974.000000,14.000000,497.000000,2.000000,23.000000,18.000000,94.000000,112.000000,1.000000,135.000000,325.000000
max,83413.000000,2021.000000,38125.000000,195.000000,1653.000000,129.000000,902.000000,1324.000000,1348.000000,4266.000000,371.000000,4266.000000,4266.000000


In [150]:
def parseUniqueIds(df):
    db = dict(df.groupby('id_driver')['dt'].apply(list))
    print(db)
def addDMYSS(df):
    
    df['dts'] = df['dt'].str.split(pat="-")
    
    df['year'] = df.apply(lambda row: row['dts'][0] ,axis=1).astype(int)
    
    df['month'] = df.apply(lambda row: row['dts'][1] ,axis=1).astype(int)
    df['sinMonth'] = np.sin(2 * np.pi * df.month / 12 ) #cyclic month 
    
    df['day'] = df.apply(lambda row: row['dts'][2] ,axis=1).astype(int)
    df['sinDay'] = np.sin(2 * np.pi * df.day / 30.43 ) #cyclic day  
    
    df['daysjan2015'] = df.apply(lambda row: row['year'] * 365 + 
                                 row['month'] * 30.43 + 
                                 row['day'] - 735506.43, axis=1)  #not quite right, but should be a good approximation 
    df = df.drop('dts',axis=1)
    df = df.drop('dt',axis=1)
    
    print(df['sinMonth'].head(),df['month'].head(),df['day'].head(),df['sinDay'].head(),df['daysjan2015'].head())

In [151]:
##parseUniqueIds(df)
addDMYSS(df)

0   -2.449294e-16
1    5.000000e-01
2   -2.449294e-16
3    8.660254e-01
4   -5.000000e-01
Name: sinMonth, dtype: float64 0    12
1     1
2    12
3     2
4     7
Name: month, dtype: int32 0    16
1    15
2    26
3    10
4    24
Name: day, dtype: int32 0   -0.161378
1    0.044379
2   -0.792383
3    0.880442
4   -0.970589
Name: sinDay, dtype: float64 0    1809.73
1    2204.00
2    1819.73
3    2229.43
4     935.58
Name: daysjan2015, dtype: float64
